In [1]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
import ast

# Training and Testing Multinomial Naive Bayes Classifier

## Reading in Training Data and Simple Transforms

In [2]:
train = pd.read_csv('./data/processed/goodreads_books_train_processed.csv',
                    converters={'cleaned_descriptions':ast.literal_eval},
                    index_col=0)
train.head()

,is_english_description,cleaned_descriptions,genre_1,votes_1
index,,,,
0,True,"[filmmak, chase, ryan, keith, ellison, complet...",Fiction,174.0
2,True,"[incred, nation, bestsel, chang, peopl, live, ...",Self Help,1162.0
3,True,"[high, spirit, intellig, good, look, abigail, ...",Romance,550.0
4,True,"[news, went, sylvest, rayn, eleg, impecc, duke...",Romance,725.0
5,True,"[major, motion, pictur, star, nicola, cage, di...",Gothic,106.0


Scikit-learn has the ability to do all of the text cleaning we have done already EXCEPT stemming, so I'll use our cleaned descriptions after stitching them back together such that each description is a string.

We are passing counts of unique words as features, meaning that the dimensions of `X_train` are `(number of descriptions, number of unique words)`.  The targets in `y_train` are just strings representing each genre.

In [3]:
train['mod_descriptions'] = train['cleaned_descriptions'].apply(lambda x: ' '.join(x))

valid_train = train[train[['mod_descriptions', 'genre_1']].notnull().all(axis=1)]

vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(valid_train.mod_descriptions)
y_train = valid_train['genre_1']

classifier = MultinomialNB()
classifier.fit(X_train, y_train)

MultinomialNB()

## Read in Validation Data

In [4]:
val = pd.read_csv('./data/processed/goodreads_books_val_processed.csv',
                   converters={'cleaned_descriptions':ast.literal_eval},
                   index_col=0)
val.head()

,is_english_description,cleaned_descriptions,genre_1,votes_1,genre_2,votes_2,genre_3,votes_3,genre_4,votes_4,...,genre_16,votes_16,genre_17,votes_17,genre_18,votes_18,genre_19,votes_19,genre_20,votes_20
index,,,,,,,,,,,,,,,,,,,,,
0,True,"[1968, lynni, young, white, woman, development...",Romance,467.0,Historical,467.0,Fiction,467.0,Adult Fiction,467.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,True,"[miranda, wake, alon, park, bench, memori, pan...",Young Adult,276.0,Science Fiction,276.0,Fantasy,276.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,True,"[âmotherâ, dayâ, part, one, talia, al, g...",Superheroes,13.0,Sequential Art,13.0,Comics,13.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,True,"[asin, b00dh4ssy6, ha, move, latest, edit, onc...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,True,"[ladi, evelynâ, life, chang, mother, die, st...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Make Similar Transforms and Score

In [5]:
# temp_descriptions = cleaned_descriptions_val.sample(n=100)
y_val = []

for rownum, row in val.filter(like='genre').iterrows():
    y_val.append(row[row.notnull()].to_list())

val_descriptions = val['cleaned_descriptions'].apply(lambda x: ' '.join(x))
X_val = vectorizer.transform(val_descriptions)

In [8]:
np.sort(classifier.predict_proba(X_val[0]))

In [16]:
classifier.classes_[:3][np.argsort([2,3,1])[::-1]]

array(['Childrens', 'Animals', 'Christian Fiction'], dtype='<U28')

In [ ]:
rbo_list = []
predictions = []
p = 0.75

for genre_list, description in tqdm(zip(val_genres, val_descriptions), total=len(val_genres)):
    result = classifier.predict_proba(description)
    predicted_genres = classifier.classes_[np.argsort(result)[::-1]]
    if len(predicted_genres)>0 and len(genre_list)>0:
        predictions.append(predicted_genres)
        rbo_list.append(rbo(predicted_genres, genre_list, p))

## Old Scoring Below

In [5]:
val['mod_descriptions'] = val['cleaned_descriptions'].apply(lambda x: ' '.join(x))
valid_val = val[val[['mod_descriptions', 'genre_1']].notnull().all(axis=1)]
X_val = vectorizer.transform(valid_val.mod_descriptions)
y_val = valid_val[['genre_1']]

In [6]:
classifier.score(X_val,y_val)

0.4371867979955072

The score above is actually not bad considering how overlapping the genres are... It could probably be greatly improved just by making genres more distinct.